In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directories for training and validation datasets
train_dir = '/path/to/dataset/train'
validation_dir = '/path/to/dataset/validation'

# Data augmentation for training
training_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescale for validation
validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow from directory
train_generator = training_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=64,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=64,
    class_mode='categorical'
)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=30, steps_per_epoch=8, validation_data=validation_generator, verbose=1, validation_steps=3)

# Save the model
model.save('/kaggle/working/saved_model.h5')

# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation Loss')
plt.legend(loc=0)
plt.figure()

plt.show()

# Load the model
model = tf.keras.models.load_model('/kaggle/working/saved_model.h5')

def preprocess_image(img_path):
    img = tf.keras.utils.load_img(img_path, target_size=(150, 150))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale as done during training
    return img_array

def classify_image(model, img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions, axis=1)[0]
    
    # Map class indices to disease types
    class_names = ['Alternaria', 'Anthracnose', 'Black Mould Rot', 'Healthy', 'Stem end Rot']
    disease_type = class_names[class_idx]
    
    return disease_type, predictions

# Example path from the dataset
img_path = '/path/to/dataset/Alternaria/sample_image.jpg'  # Adjust as necessary

# Check if the path exists
if os.path.exists(img_path):
    # Classify the image
    disease_type, predictions = classify_image(model, img_path)

    print(f'The image is classified as: {disease_type}')
    print(f'Prediction probabilities: {predictions}')

    # Display the image
    img = tf.keras.utils.load_img(img_path)
    plt.imshow(img)
    plt.title(f'Classified as: {disease_type}')
    plt.axis('off')
    plt.show()
else:
    print("Image path does not exist. Please check the path and try again.")


ModuleNotFoundError: No module named 'tensorflow'